In [1]:
from netCDF4 import Dataset
import pandas as pd

In [10]:
nc_file = Dataset('../app/data/cmems_mod_glo_bgc-pft_anfc_0.25deg_P1D-m_1710340033733.nc', 'r')

In [11]:
nc_file.variables.keys()

dict_keys(['chl', 'depth', 'latitude', 'longitude', 'phyc', 'time'])

In [12]:
nc_file.variables['phyc']

<class 'netCDF4._netCDF4.Variable'>
float32 phyc(time, depth, latitude, longitude)
    units: mmol m-3
    _FillValue: nan
    standard_name: mole_concentration_of_phytoplankton_expressed_as_carbon_in_sea_water
    long_name: Total Phytoplankton
unlimited dimensions: 
current shape = (439, 11, 9, 11)
filling off

In [13]:
nc_file.variables['depth']

<class 'netCDF4._netCDF4.Variable'>
float32 depth(depth)
    standard_name: depth
    long_name: Depth
    units: m
    unit_long: Meters
    axis: Z
    positive: down
    valid_min: 0.49402538
    valid_max: 5727.9165
unlimited dimensions: 
current shape = (11,)
filling off

In [14]:
nc_file = Dataset('../app/data/cmems_mod_glo_phy-thetao_anfc_0.083deg_P1D-m_1710340018462.nc', 'r') 

temperature = []
for idx_time, time in enumerate(nc_file.variables['time'][:]):
    for idx_depth, depth in enumerate(nc_file.variables['depth'][:]):
        for idx_lat, lat in enumerate(nc_file.variables['latitude'][:]):
            for idx_lon, lon in enumerate(nc_file.variables['longitude'][:]):
                temperature.append({
                'time': time,
                'depth': depth,
                'latitude': lat,
                'longitude': lon,
                'temperature': nc_file.variables['thetao'][idx_time, idx_depth, idx_lat, idx_lon],
                    })
temperature = pd.DataFrame(temperature)
temperature['time'] = pd.to_datetime(temperature['time'], unit='s')

In [39]:
lat, lon = temperature.sample(1)[['latitude', 'longitude']].values[0]
temperature

,time,depth,latitude,longitude,temperature
0,2023-01-01,1.541375,-41.0,-75.999992,15.721602
1,2023-01-01,1.541375,-41.0,-75.916664,15.770151
2,2023-01-01,1.541375,-41.0,-75.833328,15.80881
3,2023-01-01,1.541375,-41.0,-75.749992,15.842848
4,2023-01-01,1.541375,-41.0,-75.666664,15.890457
...,...,...,...,...,...
3605245,2024-03-12,18.495560,-39.0,-73.916664,14.503203
3605246,2024-03-12,18.495560,-39.0,-73.833328,14.797481
3605247,2024-03-12,18.495560,-39.0,-73.749992,14.535657
3605248,2024-03-12,18.495560,-39.0,-73.666664,13.640317


In [41]:
import geopandas as gpd

In [42]:
ruta = (gpd.read_file('../app/data/Track_OOO.gpx', layer='tracks')).explode().reset_index(drop=True)
ruta = ruta.set_crs('epsg:4326')
ruta['text'] = ruta.name.apply(lambda x: f'ruta {x}')

/var/folders/s2/lnzc2s7j3t9ch7f10ppkfc5x4_fqxh/T/ipykernel_42045/2849959275.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  ruta = (gpd.read_file('../app/data/Track_OOO.gpx', layer='tracks')).explode().reset_index(drop=True)


In [46]:
5%3

2

In [44]:
ruta.to_file('../app/data/ruta.geojson', driver='GeoJSON')

In [2]:
def load_temperature():

    nc_file = Dataset('../app/data/cmems_mod_glo_phy-thetao_anfc_0.083deg_P1D-m_1710340018462.nc', 'r') 
    
    temperature = []
    for idx_time, time in enumerate(nc_file.variables['time'][:]):
        for idx_depth, depth in enumerate(nc_file.variables['depth'][:]):
            for idx_lat, lat in enumerate(nc_file.variables['latitude'][:]):
                for idx_lon, lon in enumerate(nc_file.variables['longitude'][:]):
                    temperature.append({
                    'time': time,
                    'depth': depth,
                    'latitude': lat,
                    'longitude': lon,
                    'temperature': nc_file.variables['thetao'][idx_time, idx_depth, idx_lat, idx_lon],
                     })
    temperature = pd.DataFrame(temperature)
    temperature['time'] = pd.to_datetime(temperature['time'], unit='s')
    temperature = temperature.query('temperature > -100')
    temperature['temperature'] = temperature['temperature'].astype(float)
    
    return temperature


def load_chlorophyll():
    
    nc_file = Dataset('../app/data/cmems_mod_glo_bgc-pft_anfc_0.25deg_P1D-m_1710340033733.nc', 'r')
    
    chlorophyll = []
    for idx_time, time in enumerate(nc_file.variables['time'][:]):
        for idx_depth, depth in enumerate(nc_file.variables['depth'][:]):
            for idx_lat, lat in enumerate(nc_file.variables['latitude'][:]):
                for idx_lon, lon in enumerate(nc_file.variables['longitude'][:]):
                    chlorophyll.append({
                    'time': time,
                    'depth': depth,
                    'latitude': lat,
                    'longitude': lon,
                    'chlorophyll': nc_file.variables['chl'][idx_time, idx_depth, idx_lat, idx_lon],
                    'phyc': nc_file.variables['phyc'][idx_time, idx_depth, idx_lat, idx_lon],
                })
    chlorophyll = pd.DataFrame(chlorophyll)
    chlorophyll['time'] = pd.to_datetime(chlorophyll['time'], unit='s')
    chlorophyll = chlorophyll.query('chlorophyll > -100')
    chlorophyll['chlorophyll'] = chlorophyll['chlorophyll'].astype(float)
    chlorophyll['phyc'] = chlorophyll['phyc'].astype(float)
    return chlorophyll

In [3]:
chlorophyll = load_chlorophyll()
temperature = load_temperature()

/var/folders/s2/lnzc2s7j3t9ch7f10ppkfc5x4_fqxh/T/ipykernel_42045/307497982.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chlorophyll['chlorophyll'] = chlorophyll['chlorophyll'].astype(float)
/var/folders/s2/lnzc2s7j3t9ch7f10ppkfc5x4_fqxh/T/ipykernel_42045/307497982.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chlorophyll['phyc'] = chlorophyll['phyc'].astype(float)
/var/folders/s2/lnzc2s7j3t9ch7f10ppkfc5x4_fqxh/T/ipykernel_42045/307497982.py:20: SettingWithCopyWarning: 
A value is trying to be

In [5]:
temperature.to_parquet('../app/data/temperature.parquet')
chlorophyll.to_parquet('../app/data/chlorophyll.parquet')